# 4.11 Debugging With git bisect


*Estimated time to complete this notebook: 5 minutes*

You can use

``` bash
git bisect
```

to find out which commit caused a bug.

## An example repository

In a nice open source example, I found an arbitrary exemplar on github

In [1]:
import os

top_dir = os.getcwd()
git_dir = os.path.join(top_dir, "learning_git")
os.chdir(git_dir)

In [2]:
%%bash
rm -rf bisectdemo
git clone https://github.com/shawnsi/bisectdemo.git

Cloning into 'bisectdemo'...


In [3]:
bisect_dir = os.path.join(git_dir, "bisectdemo")
os.chdir(bisect_dir)

In [4]:
%%bash
python squares.py 2 # 4

4


This has been set up to break itself at a random commit, and leave you to use
bisect to work out where it has broken:

In [5]:
%%bash
./breakme.sh > break_output

error: branch 'buggy' not found.


Switched to a new branch 'buggy'


Which will make a bunch of commits, of which one is broken, and leave you in the broken final state

In [6]:
python squares.py 2 # Error message

SyntaxError: invalid syntax (2468026376.py, line 1)

## Bisecting manually

In [7]:
%%bash
git bisect start
git bisect bad # We know the current state is broken
git checkout master
git bisect good # We know the master branch state is OK

status: waiting for both good and bad commits


status: waiting for good commit(s), bad commit known


Switched to branch 'master'


Your branch is up to date with 'origin/master'.


Bisecting: 500 revisions left to test after this (roughly 9 steps)


[8f594a0dc58e989d0e6be9cf84f2093508508ae8] Comment 500


Bisect needs one known good and one known bad commit to get started

## Solving Manually

``` bash
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
python squares.py 2 # Crash
git bisect bad
python squares.py 2 # Crash
git bisect bad
python squares.py 2 # Crash
git bisect bad
python squares.py 2 #Crash
git bisect bad
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
```


And eventually:

``` bash
git bisect good
    Bisecting: 0 revisions left to test after this (roughly 0 steps)

python squares.py 2
    4

git bisect good
2777975a2334c2396ccb9faf98ab149824ec465b is the first bad commit
commit 2777975a2334c2396ccb9faf98ab149824ec465b
Author: Shawn Siefkas <shawn.siefkas@meredith.com>
Date:   Thu Nov 14 09:23:55 2013 -0600

    Breaking argument type

```

``` bash
git bisect end
```

## Solving automatically

If we have an appropriate unit test, we can do all this automatically:

In [8]:
%%bash
git bisect start
git bisect bad HEAD # We know the current state is broken
git bisect good master # We know master is good
git bisect run python squares.py 2

Previous HEAD position was 8f594a0 Comment 500


Switched to branch 'buggy'


status: waiting for both good and bad commits


status: waiting for good commit(s), bad commit known


Bisecting: 500 revisions left to test after this (roughly 9 steps)


...

Date:   Thu Nov 14 09:23:55 2013 -0600


    Breaking argument type


 squares.py | 2 +-


 1 file changed, 1 insertion(+), 1 deletion(-)


bisect found first bad commit

Boom!